<a href="https://www.kaggle.com/code/psywarrior/project-abstraction?scriptVersionId=255266137" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# ==============================================================================
# ARC Prize 2025 - Final Solver
#
# Author: Sanyam Sanjay Sharma
# Date: August 10, 2025
#
# Description:
# This script is a complete and compliant AI solver for the ARC Prize 2025.
# It operates by identifying objects within each puzzle grid and then using
# a program synthesis engine to find a sequence of transformations that
# solves the task.
#
# Core Components:
#   1. Perception Module: Identifies distinct objects in a grid.
#   2. Domain-Specific Language (DSL): A powerful set of functions for grid
#      and object manipulation (e.g., move, rotate, recolor).
#   3. Program Synthesis Engine: A true multi-step Breadth-First Search (BFS)
#      algorithm that finds sequences of operations.
#   4. Abstract Reasoning: The solver uses abstract selectors (e.g., "the
#      largest object", "all blue objects") to generalize solutions.
# ==============================================================================

import json
import os
from pathlib import Path
import numpy as np
from collections import deque
import time
import copy

In [2]:
# ==============================================================================
# SECTION 1: CORE PERCEPTION MODULE (The "Eyes")
# ==============================================================================

def find_objects(grid):
    """Identifies all distinct, contiguous objects in a grid."""
    if not isinstance(grid, np.ndarray):
        grid = np.array(grid, dtype=int)
    height, width = grid.shape
    if grid.size == 0: return []
    colors, counts = np.unique(grid, return_counts=True)
    background_color = colors[np.argmax(counts)]
    visited = np.zeros_like(grid, dtype=bool)
    objects = []
    
    for r in range(height):
        for c in range(width):
            if visited[r, c] or grid[r, c] == background_color: continue
            obj_color = grid[r, c]
            new_object = {'color': int(obj_color), 'pixels': [], 'min_row': r, 'max_row': r, 'min_col': c, 'max_col': c}
            q = deque([(r, c)])
            visited[r, c] = True
            pixels_in_obj = []
            while q:
                row, col = q.popleft()
                pixels_in_obj.append((row, col))
                new_object['min_row'] = min(new_object['min_row'], row)
                new_object['max_row'] = max(new_object['max_row'], row)
                new_object['min_col'] = min(new_object['min_col'], col)
                new_object['max_col'] = max(new_object['max_col'], col)
                for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    nr, nc = row + dr, col + dc
                    if 0 <= nr < height and 0 <= nc < width and not visited[nr, nc] and grid[nr, nc] == obj_color:
                        visited[nr, nc] = True
                        q.append((nr, nc))
            new_object['pixels'] = sorted(pixels_in_obj)
            obj_h = new_object['max_row'] - new_object['min_row'] + 1
            obj_w = new_object['max_col'] - new_object['min_col'] + 1
            shape = np.full((obj_h, obj_w), int(background_color), dtype=int)
            for r_pix, c_pix in new_object['pixels']:
                shape[r_pix - new_object['min_row'], c_pix - new_object['min_col']] = obj_color
            new_object['shape'] = shape
            new_object['size'] = len(new_object['pixels'])
            objects.append(new_object)
            
    objects.sort(key=lambda o: (o['min_row'], o['min_col']))
    for i, obj in enumerate(objects):
        obj['id'] = i
        
    return objects

In [3]:
# ==============================================================================
# SECTION 2: DOMAIN-SPECIFIC LANGUAGE (DSL) & ABSTRACT REASONING
# ==============================================================================

def select_objects(objects, selector):
    """Selects objects based on abstract properties."""
    prop = selector['property']
    val = selector['value']
    
    if prop == 'all': return objects
    if prop == 'color': return [o for o in objects if o['color'] == val]
    if prop == 'size':
        sizes = [o['size'] for o in objects]
        if not sizes: return []
        if val == 'max': target_size = max(sizes)
        elif val == 'min': target_size = min(sizes)
        else: target_size = val
        return [o for o in objects if o['size'] == target_size]
    return []

def apply_program(grid, program):
    """Applies a single transformation program to a grid."""
    grid = np.array(grid)
    program_name, params = program
    
    if program_name.startswith('obj_'):
        objects = find_objects(grid)
        selected_objects = select_objects(objects, params['selector'])
        if not selected_objects: return grid
        
        new_grid = grid.copy()
        for obj in selected_objects:
            if program_name == 'obj_recolor':
                new_grid = recolor_object(new_grid, obj, params['new_color'])
            elif program_name == 'obj_move':
                new_grid = move_object(new_grid, obj, params['dr'], params['dy'])
        return new_grid
            
    elif program_name.startswith('grid_'):
        if program_name == 'grid_rotate':
            return np.rot90(grid, k=params['k'])
        if program_name == 'grid_flip':
            return np.flip(grid, axis=params['axis'])
            
    return grid

def recolor_object(grid, obj, new_color):
    for r, c in obj['pixels']: grid[r, c] = new_color
    return grid

def move_object(grid, obj, dr, dy):
    bg = get_background_color(grid)
    # Create a temporary grid to avoid conflicts when moving overlapping objects
    temp_grid = grid.copy()
    for r, c in obj['pixels']: temp_grid[r, c] = bg
    
    h, w = grid.shape
    for r, c in obj['pixels']:
        nr, nc = r + dr, c + dy
        if 0 <= nr < h and 0 <= nc < w:
            temp_grid[nr, nc] = obj['color']
    return temp_grid

def get_background_color(grid):
    colors, counts = np.unique(grid, return_counts=True)
    return colors[np.argmax(counts)]

In [4]:
# ==============================================================================
# SECTION 3: THE PROGRAM SYNTHESIS ENGINE (The "Brain")
# ==============================================================================

def make_hashable(o):
    """Recursively converts a container to a hashable type (tuple)."""
    if isinstance(o, dict):
        return tuple(sorted((k, make_hashable(v)) for k, v in o.items()))
    if isinstance(o, (list, tuple)):
        return tuple(make_hashable(e) for e in o)
    if isinstance(o, np.ndarray):
        return o.tobytes()
    return o

def generate_candidate_programs(task):
    """Generates a list of plausible single-step abstract transformations."""
    candidate_programs = []
    
    # Grid-level operations
    for k in [1, 2, 3]: candidate_programs.append(('grid_rotate', {'k': k}))
    for axis in [0, 1]: candidate_programs.append(('grid_flip', {'axis': axis}))
    
    # Object-level selectors
    selectors = [
        {'property': 'all'},
        {'property': 'size', 'value': 'max'},
        {'property': 'size', 'value': 'min'}
    ]
    unique_colors = np.unique(np.array(task['train'][0]['input'])).tolist()
    for color in unique_colors:
        selectors.append({'property': 'color', 'value': color})

    # Generate programs for each selector
    for selector in selectors:
        for color in range(10):
            candidate_programs.append(('obj_recolor', {'selector': selector, 'new_color': color}))
        for dr in range(-5, 6):
            for dy in range(-5, 6):
                if dr != 0 or dy != 0:
                    candidate_programs.append(('obj_move', {'selector': selector, 'dr': dr, 'dy': dy}))
    
    return candidate_programs

def search_for_program(task, candidate_programs, max_depth=2, timeout_seconds=20):
    """
    Performs a Breadth-First Search (BFS) to find a sequence of transformations.
    """
    start_time = time.time()
    
    initial_states = [np.array(p['input']) for p in task['train']]
    target_states = [np.array(p['output']) for p in task['train']]
    
    # The queue stores tuples of (current_grid_states, program_sequence)
    queue = deque([(initial_states, [])])
    
    # A set to store visited states to avoid cycles. A state is the tuple of all training grids.
    visited_states = {make_hashable(initial_states)}

    while queue:
        if time.time() - start_time > timeout_seconds: return None
        
        current_states, current_program_seq = queue.popleft()

        # Check if current state is the solution
        is_solution = all(np.array_equal(cs, ts) for cs, ts in zip(current_states, target_states))
        if is_solution and current_program_seq:
            return current_program_seq

        if len(current_program_seq) >= max_depth: continue

        for program in candidate_programs:
            new_program_seq = current_program_seq + [program]
            
            try:
                next_states = [apply_program(s, program) for s in current_states]
            except Exception:
                continue

            next_states_hash = make_hashable(next_states)
            if next_states_hash in visited_states: continue
            visited_states.add(next_states_hash)
            
            queue.append((next_states, new_program_seq))
            
    return None

def solve_task(task):
    """Orchestrates the process of finding and applying a solution program."""
    candidate_programs = generate_candidate_programs(task)
    solution_program_seq = search_for_program(task, candidate_programs)
    
    predictions = []
    for test_pair in task['test']:
        predicted_grid = np.array(test_pair['input'])
        if solution_program_seq:
            for program in solution_program_seq:
                predicted_grid = apply_program(predicted_grid, program)
        predictions.append(predicted_grid.tolist())
        
    return predictions

In [5]:
# ==============================================================================
# SECTION 4: KAGGLE SUBMISSION BOILERPLATE
# ==============================================================================

if __name__ == '__main__':
    start_time_total = time.time()
    data_path = Path('/kaggle/input/arc-prize-2025')
    test_challenges_file = data_path / 'arc-agi_test_challenges.json'
    submission_file = Path('/kaggle/working/submission.json')

    if not data_path.exists():
        print("Kaggle environment not found. Creating dummy files for local testing.")
        os.makedirs('/kaggle/input/arc-prize-2025', exist_ok=True)
        os.makedirs('/kaggle/working', exist_ok=True)
        dummy_task_id = "a7495283"
        dummy_data = {dummy_task_id: {"train":[{"input":[[0,0,0,0],[0,1,1,0],[0,1,0,0],[0,0,0,0]],"output":[[0,0,0,0],[0,2,2,0],[0,2,0,0],[0,0,0,0]]}],"test":[{"input":[[0,3,3,0],[0,3,3,0],[0,0,3,0],[0,0,0,0]],"output":[[0,8,8,0],[0,8,8,0],[0,0,8,0],[0,0,0,0]]}]}}
        with open(test_challenges_file, 'w') as f: json.dump(dummy_data, f)

    try:
        with open(test_challenges_file, 'r') as f: tasks = json.load(f)
    except FileNotFoundError:
        tasks = {}

    submission = {}
    for i, (task_id, task_data) in enumerate(tasks.items()):
        print(f"Processing task {i+1}/{len(tasks)}: {task_id}")
        predicted_grids = solve_task(task_data)
        task_predictions = [{"attempt_1": grid, "attempt_2": grid} for grid in predicted_grids]
        submission[task_id] = task_predictions

    with open(submission_file, 'w') as f: json.dump(submission, f)
    
    end_time_total = time.time()
    print("-" * 30)
    print(f"Submission file created at: {submission_file}")
    print(f"Total tasks processed: {len(submission)}")
    print(f"Total runtime: {end_time_total - start_time_total:.2f} seconds")
    print("Script finished successfully.")

Processing task 1/240: 00576224
Processing task 2/240: 007bbfb7
Processing task 3/240: 009d5c81
Processing task 4/240: 00d62c1b
Processing task 5/240: 00dbd492
Processing task 6/240: 017c7c7b
Processing task 7/240: 025d127b
Processing task 8/240: 03560426
Processing task 9/240: 045e512c
Processing task 10/240: 0520fde7
Processing task 11/240: 05269061
Processing task 12/240: 05a7bcf2
Processing task 13/240: 05f2a901
Processing task 14/240: 0607ce86
Processing task 15/240: 0692e18c
Processing task 16/240: 06df4c85
Processing task 17/240: 070dd51e
Processing task 18/240: 08ed6ac7
Processing task 19/240: 09629e4f
Processing task 20/240: 0962bcdd
Processing task 21/240: 09c534e7
Processing task 22/240: 0a1d4ef5
Processing task 23/240: 0a2355a6
Processing task 24/240: 0a938d79
Processing task 25/240: 0b148d64
Processing task 26/240: 0b17323b
Processing task 27/240: 0bb8deee
Processing task 28/240: 0becf7df
Processing task 29/240: 0c786b71
Processing task 30/240: 0c9aba6e
Processing task 31/